# Attribution Queries

CoolTShirts sells shirts of all kinds, as long as they are T-shaped and cool. Recently, CTS started a few marketing campaigns to increase website visits and purchases. Using touch attribution, they’d like to map their customers’ journey: from initial visit to purchase. They can use that information to optimize their marketing campaigns. 

<img src="page_visits_schema.JPG">

## Get familiar with CoolTShirts

How many campaigns and sources does CoolTShirts use? Which source is used for each campaign?

In [ ]:
# the number of distinct campaigns
SELECT COUNT(DISTINCT utm_campaign)
FROM page_visits

# the number of distinct sources
SELECT COUNT(DISTINCT utm_source)
FROM page_visits;

# find how they are related
SELECT DISTINCT utm_source, utm_campaign
FROM page_visits;

In [ ]:
# What pages are on the CoolTShirts website?
SELECT DISTINCT page_name
FROM page_visits;

## What is the user journey?

In [ ]:
# How many first touches is each campaign responsible for?
WITH first_touch AS (
    SELECT user_id,
        MIN(timestamp) as first_touch_at
    FROM page_visits
    GROUP BY user_id),
cross_join AS
(SELECT ft.user_id,
    ft.first_touch_at,
    pv.utm_source,
		pv.utm_campaign
FROM first_touch ft
JOIN page_visits pv
    ON ft.user_id = pv.user_id
    AND ft.first_touch_at = pv.timestamp)
SELECT utm_campaign, COUNT(first_touch_at) as 'num_first_touch'
FROM cross_join
GROUP BY 1
ORDER BY num_first_touch DESC;

In [ ]:
# How many last touches is each campaign responsible for?
WITH last_touch AS (
    SELECT user_id,
        MAX(timestamp) as last_touch_at
    FROM page_visits
    GROUP BY user_id),
cross_join AS
(SELECT lt.user_id,
    lt.last_touch_at,
    pv.utm_source,
		pv.utm_campaign
FROM last_touch lt
JOIN page_visits pv
    ON lt.user_id = pv.user_id
    AND lt.last_touch_at = pv.timestamp)
SELECT utm_campaign, COUNT(last_touch_at) as 'num_last_touch'
FROM cross_join
GROUP BY 1
ORDER BY num_last_touch DESC;

In [ ]:
# How many visitors make a purchase?
SELECT page_name, COUNT(DISTINCT user_id) as 'num_visitors_purchase'
FROM page_visits
WHERE page_name = '4 - purchase';

In [ ]:
# How many last touches on the purchase page is each campaign responsible for?
WITH last_touch AS (
    SELECT user_id,
        MAX(timestamp) as last_touch_at
    FROM page_visits
  	WHERE page_name = '4 - purchase'
    GROUP BY user_id),
cross_join AS
(SELECT lt.user_id,
    lt.last_touch_at,
    pv.utm_source,
		pv.utm_campaign
FROM last_touch lt
JOIN page_visits pv
    ON lt.user_id = pv.user_id
    AND lt.last_touch_at = pv.timestamp)
SELECT utm_campaign, COUNT(last_touch_at) as 'num_last_touch_on_purchase'
FROM cross_join
GROUP BY 1
ORDER BY num_last_touch_on_purchase DESC;